# Punto 5

## Consigna

Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ 

es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de
$n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
queda a preferencia del alumno cuál usar.

## Desarrollo

Para la demostración, se tomará cada lado de la igualdad para llevarla a una expresión tal que la equivalencia sea evidente.

### Lado izquierdo

Téngase la diagonal de una matriz resultante de un producto matricial, esto es,
$$
diag(A \cdot B) = diag(C) = D
$$ 

Por definición de diagonal principal, $C$ debe ser cuadrada. De allí, podemos decir que $C$ es de dimensión $n\times n$. Además, para que el producto $A\cdot B$ sea conforme, entonces $A$ es de $n\times p$ y $B$ es de $p\times n$. Por lo cual, sabiendo las dimensiones de la matriz que necesitamos para calcular su diagonal y cumpliendo con la conformidad del producto matricial, con seguridad decimos que $C_{n\times n} = A_{n\times p} \cdot B_{p\times n}$

Ahora, $C_{n\times n} = [c_{ij}]$, donde $i,j \in \{1,\dots, n\}$. Por definición de producto matricial, cada elemento está definido por
$$
c_{ij} = \sum_{k=1}^p a_{ik} b_{kj}
$$

Si buscamos $diag(C)$, entonces solo estamos interesados en aquellos elementos de $C$ en donde $i=j$. Si definimos $D=diag(C)$, sabemos que $D$ es un vector en $\R^n$, por lo que $D=[d_i]\quad \forall i \in \{1,\dots,n\}$. Con esto en mente, cada elemento de $D$ es
\begin{equation}
d_i = c_{ii} = \sum_{k=1}^p a_{ik} b_{ki}
\end{equation}

### Lado derecho

A partir del razonamiento anterior, sabemos que $A_{n\times p} = [a_{ij}]$ y $B_{p\times n} = [b_{ij}]$. Si transponemos $B$, entonces por definición $B^T_{n\times p} = [b'_{ij}] = [b_{ji}]$.

Definamos el [producto de Hadamard](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)) como la multiplicación elemento a elemento entre dos matrices de iguales dimensiones, en este caso $n\times p$. Así, $H_{n\times p}=A\odot B^T$, donde $H_{n\times p}=[h_{ij}]$ tal que
$$
h_{ij​} = a_{ij​}\cdot b_{ji}
$$

La expresión de la consigna exigía que $\sum_{cols} A \odot B^T$. En otras palabras, queremos sumar todos los $p$ elementos en cada i-ésima fila. Con ello obtenemos un vector $V \in \R^n$ que se compone de

\begin{equation}
\tag{2}
v_i = \sum^p_{j=1} h_{ij} = \sum^p_{k=1} a_{ik}b'_{ik} = \sum^p_{k=1} a_{ik}b_{ki}
\end{equation}

### Conclusión

Del lado izquierdo, tenemos $D_{i} = [d_i] = diag(A\cdot B)$; mientras que del derecho, $V_i = [v_i] = \sum_{cols}  A\odot B^T$ (donde $i \in \{1,\dots,n\}$). Entonces, por la ecuación 1 y 2 se sabe que los elementos de $D$ y de $V$ son equivalentes. En términos simbólicos, cada elemento resultante de ambos lados de la expresión está dado por

$$
v_i = d_i = \sum^p_{k=1} a_{ik}b_{ki} 
$$

Con esto, $diag(A \cdot B) = \sum_{cols} A \odot B^T$ porque cada elemento del vector resultante posee los mismos elementos, demostrando lo requerido en la consigna.

## Extra: Pequeño Benchmarking

Se realiza un benchmark para mostrar que "esquivar" la matriz $n\times n$ usando matrices de $n\times p$ representa un aumento en la eficiencia computacional.

In [33]:
import numpy as np
from numpy import random as rnd

In [34]:
class Benchmarking:
    """
    Una clase que usamos para realizar el benchmarking.

    Al inicializar, genera dos matrices A y B de tamaño nxp y pxn,
    respectivamente. Luego, puedo realizar el cálculo de forma ineficiente con
    el método `.inefficient()`, o eficientemente con `.efficient()`

    Argumentos:
    - n: Entero positivo para las filas de A y las columnas de B.
    - p: Entero positivo para las columnas de A y las filas de B.
    - random_seed (opcional): Si le otorgo un valor, genero A y B con una
    semilla predefinida.
    """
    def __init__(self, n, p, random_seed=None):
        if random_seed is not None:
            rnd.seed(random_seed)
        # Genero las matrices A y B con números aleatorios
        self.A = rnd.rand(n,p)
        self.B = rnd.rand(p,n)

    def inefficient(self):
        return np.diag(self.A @ self.B) # Lado izquierdo de la igualdad

    def efficient(self):
        return np.sum(self.A * self.B.T, axis=1) # Lado derecho de la igualdad

Para inicializar, utilizo números grandes a modo que la diferencia en eficiencia sea evidente: $n = 10.000$ y $p=10$. 

**Opcional:** Fijamos la semilla en 123 para asegurar reproducibilidad, pero podría omitirse o cambiarse en caso de querer experimentar con otros números.

In [35]:
# Inicializo la clase
calc = Benchmarking(n=10000, p=10, random_seed=123)

Hago los cálculos, primero de forma ineficiente y luego de la manera eficiente.

In [36]:
%%timeit
calc.inefficient()

1.07 s ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
calc.efficient()

650 μs ± 9.41 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


En promedio, el método ineficiente tardó ~1 segundo por loop, mientras que el eficiente le tomó tan solo 650 microsegundos computar cada iteración.